In [ ]:
using Polyhedra
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
import MechanismComplementarity
using JuMP, Gurobi

In [ ]:
reload("MechanismComplementarity")
mc = MechanismComplementarity

In [ ]:
function planar_base()
    world = RigidBody{Float64}("world")
    mechanism = Mechanism(world; gravity=SVector(0, 0, -9.81))

    frame = CartesianFrame3D("dummy")
    inertia = SpatialInertia(frame, 0 * eye(3), zeros(3), 0.0)
    dummy = RigidBody(inertia)
    base_x = Joint("base_x", Prismatic([1., 0, 0]))
    attach!(mechanism, world, base_x, eye(Transform3D, frame_before(base_x), default_frame(world)), dummy)

    frame = CartesianFrame3D("base")
    inertia = SpatialInertia(frame, 0.1 * eye(3), zeros(3), 1.0)
    base = RigidBody(inertia)
    base_z = Joint("base_z", Prismatic([0., 0, 1]))
    attach!(mechanism, dummy, base_z, eye(Transform3D, frame_before(base_z), default_frame(dummy)), base)
    mechanism, base
end

function planar_revolute_base()
    mechanism, base = planar_base()
    frame = CartesianFrame3D("base_revolute")
    inertia = SpatialInertia(frame, 0.1 * eye(3), zeros(3), 1.0)
    body = RigidBody(inertia)
    joint = Joint("base_rotation", Revolute([0., 1., 0]))
    attach!(mechanism, base, joint, eye(Transform3D, frame_before(joint), default_frame(base)), body)
    mechanism, body
end

In [ ]:
urdf_mech = parse_urdf(Float64, "box_valkyrie.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

vis = Visualizer()[:boxval]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))
inspect(mechanism, vis)

In [ ]:
floor = mc.Obstacle(
    default_frame(world),
    SimpleHRepresentation{3, Float64}([0 0 1], [0]),
    HalfSpace{3, Float64}([0, 0, 1], 0),
    1.0)

free_space = mc.FreeRegion(
    default_frame(world),
    SimpleHRepresentation{3, Float64}(
        [0 0 -1; 
         -1 0 0],
        [0, 1])
    )

contact_limbs = findbody.(mechanism, ["rh", "lh", "rf", "lf"])

wall = mc.Obstacle(
    default_frame(world),
    SimpleHRepresentation{3, Float64}([1 0 0], [0]),
    HalfSpace{3, Float64}([1, 0, 0], 0),
    1.0)
    

env = mc.Environment(
    Dict([body => mc.ContactEnvironment(
                [Point3D(default_frame(body), SVector(0., 0, 0))],
                [floor],
                [free_space])
            for body in contact_limbs]));

In [ ]:
limits = Dict([
    findjoint(mechanism, "core_to_rh_x") => SimpleHRepresentation([1. -1]', [0.75, -0.25]),
    findjoint(mechanism, "core_to_lh_x") => SimpleHRepresentation([1. -1]', [0.75, -0.25]),
    findjoint(mechanism, "core_to_rh_z") => SimpleHRepresentation([1. -1]', [0.5, 0.5]),
    findjoint(mechanism, "core_to_lh_z") => SimpleHRepresentation([1. -1]', [0.5, 0.5]),
    findjoint(mechanism, "core_to_rf_x") => SimpleHRepresentation([1. -1]', [0.5, 0.0]),
    findjoint(mechanism, "core_to_lf_x") => SimpleHRepresentation([1. -1]', [0.5, 0.0]),
    findjoint(mechanism, "core_to_rf_z") => SimpleHRepresentation([1. -1]', [-0.5, 1.0]),
    findjoint(mechanism, "core_to_lf_z") => SimpleHRepresentation([1. -1]', [-0.5, 1.0]),
        ])

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1.1])
set_velocity!(x0, findjoint(mechanism, "base_x"), [2.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-0.9])

In [ ]:
settransform!(vis, x0)

In [ ]:

controller = x -> begin
    kp = 50
    kd = 0.1 * kp
    u = kp .* (configuration(x0) .- configuration(x)) .- kd .* velocity(x)
    u[1:3] .= 0
    u
end

Δt = 0.05
results = mc.simulate(x0, controller, limits, env, Δt, 10, GurobiSolver(OutputFlag=0));

In [ ]:
results[end].joint_contacts[findjoint(mechanism, "core_to_rf_x")]

In [ ]:
results[end].joint_contacts[findjoint(mechanism, "core_to_rf_z")]

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-1])

Δt = 0.05
input_limits = Dict([
        joint=>[-20, 10]
    for joint in keys(limits)])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 8, Model(solver=GurobiSolver(TimeLimit=60)))
@objective model Min -100 * configuration(results_opt[end].state, findjoint(mechanism, "base_x"))[end] + sum([0.001 * sum(r.input.^2) for r in results_opt])

solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [0])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(x0, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_rf_z"), [-1])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(x0, findjoint(mechanism, "core_to_lf_z"), [-1])

Δt = 0.05
input_limits = Dict([
        joint=>[-20, 10]
    for joint in keys(limits)])

model, results_opt = mc.optimize(x0, input_limits, limits, env, Δt, 32, Model(solver=GurobiSolver(TimeLimit=120)))

objective = (
    -1 * configuration(results_opt[end].state, findjoint(mechanism, "base_x"))[end]
#     + 100 * velocity(results_opt[end].state, findjoint(mechanism, "base_x"))[end]^2
#     + 100 * velocity(results_opt[end].state, findjoint(mechanism, "base_z"))[end]^2
    )
# objective += sum([0.01 * sum(r.input.^2) for r in results_opt])

@objective model Min objective

for r in results_opt
    for (body, crs) in r.contacts
        for contact_result in crs
            JuMP.fix(contact_result.λ, 0)
        end
    end
    
    for (joint, jrs) in r.joint_contacts
        for joint_result in jrs
            JuMP.fix(joint_result.λ, 0)
        end
    end 
end

solve(model)
results_opt = getvalue.(results_opt);

In [ ]:
for r in results_opt
    settransform!(vis, r.state)
    sleep(Δt)
end

In [ ]:
i = 1
controller = x -> begin
    global i
    if i <= length(results_opt)
        i += 1
        results_opt[i - 1].input
    else
        kp = 20
        kd = 0.1 * kp
        u = kp .* (configuration(x0) .- configuration(x))[3:end] .- kd .* velocity(x)[3:end]
        vcat(zeros(2), u)
    end
        
end

results = mc.simulate(x0, controller, limits, env, Δt, 60, GurobiSolver(OutputFlag=0));

In [ ]:
settransform!(vis, results[1].state)

In [ ]:
for r in results
    settransform!(vis, r.state)
    sleep(Δt)
end